In [1]:
%%writefile ".env"

PROJECT_ID="feux_cq"

COMMUN_PATH = "C:/"
PATH_INFOCENTRE_APP = ${COMMUN_PATH}Informatique/SIG/Application/Jupyterhub/
PATH_ETUDE = ${COMMUN_PATH}Informatique/SIG/Donnees/Oeil/Traitement_Donnees/SURFACES_BRULEES_SENTINEL/2022/220502_PreparationFeu2020/
PROJECT_PATH =${COMMUN_PATH}Users/oriane.bruyere/Documents/Projet_feux/Script/Step1_detection_error/
DATA_CATALOG_DIR = ${PATH_INFOCENTRE_APP}projets/catalogFiles/ 
DATA_OUTPUT_DIR = ${PROJECT_PATH}output/
SIG_DATA_PATH = ${COMMUN_PATH}Informatique/SIG/Donnees/
DB_USER="jfnguyenvansoc"
DB_PWD="oeil"
DB_HOST="172.20.12.13"
DB_PORT=5432

DB_WORKSPACE="oeil_traitement"
DB_REF="oeil_reference"
DB_EXT="data_externe"
DB_SCHEMA = "feux_cq"
DB_SCHEMA_REF = "feux"

Overwriting .env


## Get missing burned areas files from INSIGHT server

In [2]:
import logging
from dotenv import load_dotenv
from filecmp import dircmp
from os import listdir
import os
import filecmp
import geopandas as gpd
import pandas as pd
import fiona
from sqlalchemy import create_engine
load_dotenv()
import subprocess

## Open gpkg and add data to POSTGIS

In [3]:
DATA_TEMP ='A:/FEUX_INSIGHT/2017/'
#DATA_TEMP ='C:/Users/oriane.bruyere/Documents/file_feux/2020/'

In [4]:
fichiers_geopackages = [f for f in os.listdir(DATA_TEMP) if f.endswith('.gpkg')]
gdf_list = []

for fichier_geopackage in fichiers_geopackages:
    chemin_geopackage = os.path.join(DATA_TEMP, fichier_geopackage)
    couches = fiona.listlayers(chemin_geopackage)

    for couche in couches:
        gdf_couches = gpd.read_file(chemin_geopackage, layer=couche, driver='GPKG')
        
        if 'date_texte' in gdf_couches.columns:
            gdf_couches.rename(columns={'date_texte': 'date'}, inplace=True)

        gdf_couches['date'] = gdf_couches['date'].apply(lambda x: x[:-11] if len(x) > 8 else x)
        gdf_couches['nom'] = gdf_couches['nom'].apply(lambda x: x[:19] + x[30:] if len(x) > 30 else x)
        
        gdf_list.append(gdf_couches)

gdf = pd.concat(gdf_list, ignore_index=True, sort=False)
gdf=gdf.reset_index(drop=True)

print("Données fusionnées à partir des GeoPackages :")
print(gdf)

Données fusionnées à partir des GeoPackages :
           date                             nom       province commune  \
0      20200119  SENTINEL2x_20200119_L2A_T58KFA   Province Sud   PAITA   
1      20200104  SENTINEL2x_20200104_L2A_T58KFA   Province Sud   PAITA   
2      20200119  SENTINEL2x_20200119_L2A_T58KFA   Province Sud   PAITA   
3      20200119  SENTINEL2x_20200119_L2A_T58KFA   Province Sud    YATE   
4      20200119  SENTINEL2x_20200119_L2A_T58KFA   Province Sud    YATE   
...         ...                             ...            ...     ...   
24607  20200223  SENTINEL2x_20200223_L2A_T58KFA   Province Sud   PAITA   
24608  20200223  SENTINEL2x_20200223_L2A_T58KFA   Province Sud   PAITA   
24609  20200223  SENTINEL2x_20200223_L2A_T58KFA   Province Sud   PAITA   
24610  20200223  SENTINEL2x_20200223_L2A_T58KFA   Province Sud  DUMBEA   
24611  20200223  SENTINEL2x_20200223_L2A_T58KFB  Province Nord  CANALA   

        surface       x       y cuirasse  cuira_area    bdtopo  b

C:\Users\oriane.bruyere\AppData\Local\Temp\ipykernel_5116\1094699821.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gdf = pd.concat(gdf_list, ignore_index=True, sort=False)


## Convertir les surfaces detectees en type MULTIPOLYGONE

In [5]:
from shapely.geometry import MultiPolygon

# Vérifier si la géométrie est de type Polygon
if gdf['geometry'].geom_type[0] == 'Polygon':
    gdf['geometry'] = gdf['geometry'].apply(lambda poly: MultiPolygon([poly]))

    print(gdf)
else:
    print("La géométrie n'est pas de type Polygon.")

La géométrie n'est pas de type Polygon.


## Création de la base de donnée dans POSTGIS

In [6]:
gdf = gpd.GeoDataFrame(gdf.copy(), crs=gdf.crs, geometry=gdf['geometry'])
gdf=gdf[["nom", "province", "commune", "surface", "geometry","date"]]

In [7]:
from sqlalchemy import text
table = "surfaces_brulees_brute_control"

conex = create_engine(f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PWD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_WORKSPACE")}')
gdf.to_postgis(table, conex,schema=os.getenv("DB_SCHEMA"),if_exists='append')